In [ ]:
import pandas as pd
import pingouin as pg
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

from pandas.api.types import CategoricalDtype

## Overview

In this worksheet we will look at how to use linear models in Python.
While `pingouin` is quite usel for a lot of basic statistics, for more advanced work the `statsmodels` library can be useful.
Before jumping into linear models we will first review porforming ANOVAs with `pingouin` and then see how the same thing can be done with `statsmodels`.
We will then move onto linear regression and see that there is a close connection between ANOVA and linear regression.

### ANOVA

Slicing the data and doing individual t tests is not really the best way to deal with this problem.
We have seen how using an ANOVA can help with this problem in the previous worksheet.

Though JASP hides this, the ANOVA is really a linear model like linear regression.
The main difference is the statistical approach for fitting and testing.
The next lines will setup the linear model for us.

Let's start by loading our data and doing a two way ANOVA using `pingouin`.

As in the previous worksheet we will make sure to setup the categorical and ordinal variables.

In [ ]:
# Read the data
file_name = "../data/mouse_linear_regression.csv"
df = pd.read_csv(file_name)
# Set relevant variables to categorical/ordinal with appropriate levels and order
df["Genotype"] = df["Genotype"].astype(
    CategoricalDtype(categories=["WT", "-/-"], ordered=True)
)
df["Sex"] = df["Sex"].astype(CategoricalDtype(categories=["F", "M"], ordered=False))
df["Treatment group"] = df["Treatment group"].astype(
    CategoricalDtype(categories=["Control", "TX", "TX2"], ordered=True)
)
# Take a look a the data
df.head()

Now let's use `pingouin` to do a two way ANOVA with Genotype and Treatment Group.

In [ ]:
pg.anova(
    data=df,
    dv="Weight",
    between=["Genotype", "Treatment group"],
)

While two way ANOVAs are one to approach this analysis, we could also use multivariable linear regression.
Either would be fine here, but as more variables are included the regression becomes more appropriate.

Linear regression support in `pingouin` is a bit limited at the moment, so we will use the `statsmodels` package instead.
The `statsmodels` package has a lot of functionality, but it is not as user friendly as `pingouin`.
We will start simple with Weight as our dependent variable and Genotype and Treatment group as our indepdent variables.

The first thing we need to do is create "design" matrix.
These are a way to encode a linear model for the software to analyse.
We will use the `ols` function from `statsmodels.formula.api` subpackage, which we have imported as `smf`.
This is a helper function to create a linear model.
This allows us to use what are called R style formulas for specifying statistical models.

> Most programming language don't like dealing with characters like spaces and sometimes symbols like !.
> If the columns in your data have these symbols you can either rename them or in patsy use the Q syntax.
> Below we use this for "Treatment group" because there is a space.

In [ ]:
model = smf.ols('Weight ~ Genotype + Q("Treatment group")', data=df)
model_fit = model.fit()

Before analysing the data like we would in a regression analysis, let's consider an ANOVA with `statsmodels`.

In [ ]:
sms.anova_lm(model_fit)

This does not quite match the `pingouin` analysis because `statsmodels` does not automatically add an interaction.

We can do this by specifying it in the formula.

In [ ]:
model = smf.ols('Weight ~ Genotype + Q("Treatment group") + Genotype*Q("Treatment group")', data=df)
model_fit = model.fit()
sms.anova_lm(model_fit)

Now this lines up with the `pingoin` analysis!

The omnibus ANOVA test rejects the Null hypothesis for all terms.
We can do post hoc comparisons using the Tukey test just like with `pingoin`.
Let's take a look at Treatment group.

In [ ]:
res = sms.multicomp.pairwise_tukeyhsd(df["Weight"], df["Treatment group"])
print(res)

If you compare these results to the previous worksheet analysis using `pingouin` you will see the are the same.

### Linear regression

Let's try a linear regression now.
This is a bit simpler :)

Let's start without interaction terms.

> Note: `statsmodels` does not report the p-value for Durbin-Watson, but instead provides the Jarque-Bera statistic for model assumptions.

In [ ]:
model = smf.ols('Weight ~ Genotype + Q("Treatment group")', data=df)
model_fit = model.fit()
model_fit.summary()

We can do a qqplot of the residuals to check our assumptions.

In [ ]:
ax = sm.graphics.qqplot(model_fit.resid, line="q")

The QQ plot looks okay, not great.

Let's see what happens if we add an interaction between genotype and treatment.

In [ ]:
model = smf.ols('Weight ~ Genotype + Q("Treatment group") + Genotype*Q("Treatment group")', data=df)
model_fit = model.fit()
model_fit.summary()

First note the AIC for the model including interactions is lower, suggesting a better model.

Let's take a look at the QQ plot.

In [ ]:
ax = sm.graphics.qqplot(model_fit.resid, line="q")

This does not look great.
One cause could be that we are missing a term.

In [ ]:
model = smf.ols('Weight ~ Genotype + Q("Treatment group") + Sex + Genotype*Q("Treatment group")', data=df)
model_fit = model.fit()
model_fit.summary()

The AIC has decreased with the addition of Sex.

Let's take a look at the QQ plot.

In [ ]:
ax = sm.graphics.qqplot(model_fit.resid, line="q")

This looks a bit better.
We still have not accounted for Baseline weight yet.
Let's try incoporating that into the model.

In [ ]:
model = smf.ols('Weight ~ Genotype + Q("Treatment group") + Sex + Q("Baseline weight") + Genotype*Q("Treatment group")', data=df)
model_fit = model.fit()
model_fit.summary()

In [ ]:
ax = sm.graphics.qqplot(model_fit.resid, line="q")

Adding Baseline weight has improved the model fit again as assessed by the AIC.

Now let's take a look at the VIF to determine if there is any co-linearity.
Getting the VIF out is not as easy in `statsmodels` as JASP.
We first need access to our data matrix.
Coveniently this is stored in the `model` object as an attribute `model.exog`.

In [ ]:
model.exog

We can find out what the matrix columns represent using the `exog_names` attribute.

In [ ]:
model.exog_names

We can also convert this to a DataFrame for easier inspection.

In [ ]:
exog_df = pd.DataFrame(model.exog, columns=model.exog_names)
exog_df.head()

Getting the VIF using `statsmodels` is pretty involved.
We need to import the `variance_inflation_factor` function which takes two arguments.

1. The data matrix.
2. The index of the variable we want to get the VIF for.

I am using quite a few Python tricks to get this into a nice `pandas` Series object.

- `enumerate` is used to count the index of each column
- dictionary comprehension is used to loop through the columns and test the VIF
- I use the generated dictionary to create a Series object called `vif`

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.Series({c: variance_inflation_factor(exog_df.values, i) for i, c in enumerate(exog_df.columns)})
vif

We will ignore the VIF of the intercept, as it is uncommon to worry about it.
Baseline weight and Sex have VIFs > 5.
Let's try removing Sex and looking at the model.

In [ ]:
model = smf.ols('Weight ~ Genotype + Q("Treatment group") + Q("Baseline weight") + Genotype*Q("Treatment group")', data=df)
model_fit = model.fit()
model_fit.summary()

In [ ]:
ax = sm.graphics.qqplot(model_fit.resid, line="q")

In [ ]:
exog_df = pd.DataFrame(model.exog, columns=model.exog_names)
vif = pd.Series({c: variance_inflation_factor(exog_df.values, i) for i, c in enumerate(exog_df.columns)})
vif

The AIC of the model has improved with Sex removed.
The QQ plot also looks reasonable.
Finally, all the VIFs are less than 5 which is an improvement.

> In class we said VIF < 2.5, but 5 is sometimes used as a threshold.
> You could try removing additional variables to see if you can get the VIF down to 2.5

You can also look at the other assumption checks.
Functionality is provided in `statsmodels` but it takes a bit of work.

For more on linear regression in Python you can see this [book chapter](https://ethanweed.github.io/pythonbook/05.04-regression.html).